In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.
import re
from pandas import DataFrame
# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import ast

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.common.exceptions import NoSuchElementException

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.



In [2]:
#EJEMPLO:cojo mi csv de pelis de romance para testear la iteración por ID en imdb 

df_romance = pd.read_csv("romance_test_fase2.csv", index_col=0)
df_romance.head()

,genre,type,name,year,month,id
2045,Romance,Movie,New Strains,2023,1,tt26345526
2046,Romance,Movie,Someone You Loved,2023,2,tt26351536
2047,Romance,Movie,Zindagi Tere Naan,2023,3,tt26351098
2048,Romance,Movie,Milosc do kwadratu jeszcze raz,2023,2,tt26369131
2049,Romance,Short,Whistling Cloud: Spinnin,2023,1,tt26421477


In [3]:
lista_ids = df_romance["id"]
lista_ids.head()

2045    tt26345526
2046    tt26351536
2047    tt26351098
2048    tt26369131
2049    tt26421477
Name: id, dtype: object

In [4]:
# ABRIMOS NAVEGADOR ------------------------------------------------------

driver = webdriver.Chrome()

#Busca la web que yo quiera, en este caso IMDB
driver.get("https://www.imdb.com/")
#print("0 - Abriendo la web")

# Maximizar la ventana del navegador para mostrar la página completa
driver.maximize_window()


#print("1 - Cogiendo id del csv")

#------------------------- COMENZAMOS LOOP ---------------------------------

lista_imdb = []
for id in tqdm(lista_ids):
    lista_consumible = []
    
    try:

        driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER)
        sleep(4)

        lista_consumible.append(id)
        #print(" 3- Añadiendo ID")

#PUNTUACION ----------------------------------------------------------------
        try: 
            puntuacion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-3a4309f8-0.fjtZsE.sc-dffc6c81-1.fJrHDo > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.gtEgaf > div.sc-bde20123-2.gYgHoj > span.sc-bde20123-1.iZlgcd").text
            #print("4 - Sacando puntuación imdb")
            lista_consumible.append(puntuacion)
            sleep(4)
        except: 
            lista_consumible.append("NA")
            

#DIRECCION ----------------------------------------------------------------
        try: 

            director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text
            lista_consumible.append(director)
            #print("4 - Sacando Director")
            sleep(4)
        except: 
            lista_consumible.append("NA")
#GUIONISTAS --------------------------------------------------------------
        
        try: 
            guionistas = []

            for guionista in range(1,10):
                try: 
                    nombres_guionistas = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li:nth-child({guionista}) > a").text
                    guionistas.append(nombres_guionistas)
                except:
                    break

            lista_consumible.append(guionistas)
        except: 
            lista_consumible.append("NA")
            #print("5 - Sacando Guionistas")

#ARGUMENTO PELICULA -----------------------------------------------------
        try:

            #unas pelis van con este:
            argumento = driver.find_element("xpath", "/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]").text
            #otras pelis van con este: 
            #argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.ecgcFy > div.sc-e226b0e3-6.hfusNC > div.sc-e226b0e3-10.jcOwsU > section > p > span.sc-466bb6c-2.eVLpWt").text
        except NoSuchElementException:
            # Handle the case where the element is not found
            argumento = "Element not found"

        lista_consumible.append(argumento)

        sleep(4)

        #print("9 - Sacando Argumento")


#DURACION ---------------------------------------------------------------
        try: 
            duracion_horas = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(2)").text 
            #print("6 - Sacando duración")
            #PARA CONVERTIR LA DURACION TOTAL A MINUTOS! 
            partes = duracion_horas.split() 

            horas = 0 
            minutos = 0 

            for parte in partes:
                if "h" in parte: 
                    horas = horas + int(parte[:-1]) 
                elif "m" in parte: 
                    minutos = minutos + int(parte[:-1])# :-1 al eliminar el H y el M entonces nos da el digit. Si pongo [0]me daría solo el 2 de 22, o el 2 de 25.

            duracion = horas * 60 + minutos 

            lista_consumible.append(duracion)
            sleep(6)
            #print("7 - convirtiendo a minutos")
        except: 
            lista_consumible.append("NA")

#NOMBRE PELICULA ----------------------------------------------------------------
        try: 

            nombre_imdb = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > h1 > span").text
            sleep(4)
            lista_consumible.append(nombre_imdb)
            
            #print("8 - Sacando nombre Peli")
            sleep(4)
        except:
            lista_consumible.append("NA")

#----------------------------------------------------------------

    except NoSuchElementException:
        # Cuando el ID que estamos buscando no existe en imdb
        print(f"ID {id} does not exist on IMDb. Skipping...")
        driver.back()
        sleep(8)



# Fuera del loop: 

    if lista_consumible: #evita hacer append de tuplas vacías
                            
        lista_imdb.append(tuple(lista_consumible))


 25%|██▌       | 3/12 [01:36<04:11, 27.89s/it]

ID tt26369131 does not exist on IMDb. Skipping...


100%|██████████| 12/12 [06:49<00:00, 34.08s/it]


In [7]:
df_imdb = pd.DataFrame(lista_imdb, columns=["ID", "Puntuación", "Dirección", "Guionistas", "Argumento", "Duración", "Nombre imdb"])

In [8]:
df_imdb

,ID,Puntuación,Dirección,Guionistas,Argumento,Duración,Nombre imdb
0,tt26345526,7.5,Prashanth Kamalakanthan,"[Prashanth Kamalakanthan, Artemis Shaw]","As a mysterious pandemic emerges, a squabbling...",78,New Strains
1,tt26351536,6.4,Seul-woo Hyung,[Seul-woo Hyung],Joon Ho and Ah Yeong have been best friends si...,103,Someone You Loved
2,tt26351098,NA,NA,[],Element not found,NA,NA
3,tt26421477,NA,Mun Chee Yong,[Mun Chee Yong],Element not found,NA,Whistling Cloud: Spinnin
4,tt26423913,7.1,Khaled Khella,[Khaled Khella],A conflicted young man and his overexcited gir...,30,Public Indecency
5,tt26425315,NA,Vaughn Arterberry,[Vaughn Arterberry],A man and woman discover a deeper connection b...,5,Coming of Age: Love
6,tt26438468,5.8,Dmitry Nevzorov,"[Nikita Lundin, Dmitry Nevzorov]",The story of two opposites who accidentally me...,102,Svobodnyye otnosheniya
7,tt26440619,7.3,Christian Petzold,[Christian Petzold],A group of friends are in a holiday home by th...,0,Afire
8,tt26446577,6.8,Graeme Campbell,[Anne Hollister],An aspiring marketing designer's favorite fost...,NA,Mr. Pawsitively Perfect
9,tt26446979,5.3,Vishal Bhardwaj,"[Vishal Bhardwaj, Jyotsna Hariharan]",A young man unearths an ancient relic that let...,30,Fursat
